In [1]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [2]:
input_query = inputs[1]
input_1 = inputs[0]

print(input_query)

tensor([0.5500, 0.8700, 0.6600])


In [3]:
res = 0.

for idx, ele in enumerate(input_1):
  res+= ele * input_query[idx]

print(res)

tensor(0.9544)


In [4]:
i = 0

print(torch.dot(inputs[i], input_query))

tensor(0.9544)


In [5]:
query = inputs[1]

attn_score2 = torch.empty(inputs.shape[0])

for ind, val in enumerate(inputs):
  attn_score2[ind] = torch.dot(val, query)


print(attn_score2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [6]:
attn_weight = attn_score2 / attn_score2.sum()

attn_weight

tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])

In [7]:
torch.softmax(attn_score2, dim= 0)

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

In [8]:
contextvec = torch.zeros(query.shape)

for ind, val in enumerate(inputs):
  contextvec += attn_weight[ind] * inputs[ind]

print(contextvec)

tensor([0.4355, 0.6451, 0.5680])


simple self attention without trianable weights

In [9]:
query = inputs[1]

attn_score2 = torch.empty(inputs.shape[0])

for ind, val in enumerate(inputs):
  attn_score2[ind] = torch.dot(val, query)


print(torch.softmax(attn_score2, dim=0).sum())

tensor(1.)


In [10]:
#matrix mutltplication


attn_score = inputs @ inputs.T
attn_weight = torch.softmax(attn_score, dim=1)
contextvec = attn_weight @ inputs


print(attn_score)
print(attn_weight)
print(attn_weight.sum(dim=1))
print(contextvec)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])
tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])
tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])
tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [11]:
x =  inputs[1]

d_in = inputs.shape[1]
d_out = 2

torch.manual_seed(123)

w_query = torch.nn.Parameter(torch.randn(d_in, d_out))
w_key = torch.nn.Parameter(torch.randn(d_in, d_out))
w_value = torch.nn.Parameter(torch.randn(d_in, d_out))

print(w_query)
print(w_key)
print(w_value)

Parameter containing:
tensor([[-0.1115,  0.1204],
        [-0.3696, -0.2404],
        [-1.1969,  0.2093]], requires_grad=True)
Parameter containing:
tensor([[-0.9724, -0.7550],
        [ 0.3239, -0.1085],
        [ 0.2103, -0.3908]], requires_grad=True)
Parameter containing:
tensor([[ 0.2350,  0.6653],
        [ 0.3528,  0.9728],
        [-0.0386, -0.8861]], requires_grad=True)


In [12]:
quer2 = x @ w_query
key = inputs @ w_key
value = inputs @ w_value

print(quer2)
print(key)
print(value)

tensor([-1.1729, -0.0048], grad_fn=<SqueezeBackward4>)
tensor([[-0.1823, -0.6888],
        [-0.1142, -0.7676],
        [-0.1443, -0.7728],
        [ 0.0434, -0.3580],
        [-0.6467, -0.6476],
        [ 0.3262, -0.3395]], grad_fn=<MmBackward0>)
tensor([[ 0.1196, -0.3566],
        [ 0.4107,  0.6274],
        [ 0.4091,  0.6390],
        [ 0.2436,  0.4182],
        [ 0.2653,  0.6668],
        [ 0.2728,  0.3242]], grad_fn=<MmBackward0>)


In [13]:
attn_score_2 = quer2 @ key.T

attn_score_2

tensor([ 0.2172,  0.1376,  0.1730, -0.0491,  0.7616, -0.3809],
       grad_fn=<SqueezeBackward4>)

In [14]:
d_k = key.shape[1]

attn_weight_2 = torch.softmax(attn_score_2 / (d_k ** 0.5), dim=-1)

attn_weight_2

tensor([0.1704, 0.1611, 0.1652, 0.1412, 0.2505, 0.1117],
       grad_fn=<SoftmaxBackward0>)

In [15]:
context_vec_2 = attn_weight_2 @ value

context_vec_2

tensor([0.2854, 0.4081], grad_fn=<SqueezeBackward4>)

In [16]:
import torch.nn as nn

class SelfAttentionv1(nn.Module):
  def __init__(self, d_in, d_out):
    super().__init__()
    self.w_query = torch.nn.Parameter(torch.randn(d_in, d_out))
    self.w_key = torch.nn.Parameter(torch.randn(d_in, d_out))
    self.w_value = torch.nn.Parameter(torch.randn(d_in, d_out))

  def forward(self, x):

    query = inputs @ self.w_query
    key = inputs @ self.w_key
    value = inputs @ self.w_value

    attn_score = query @ key.T
    attn_weights = torch.softmax(attn_score / (key.shape[1] ** 0.5), dim=-1)
    context_vec = attn_weights @ value

    return context_vec

torch.manual_seed(123)

s1 = SelfAttentionv1(d_in, d_out)

s1(inputs)

tensor([[0.2845, 0.4071],
        [0.2854, 0.4081],
        [0.2854, 0.4075],
        [0.2864, 0.3974],
        [0.2863, 0.3910],
        [0.2860, 0.4039]], grad_fn=<MmBackward0>)

# self attention

In [17]:
import torch.nn as nn

class SelfAttentionv2(nn.Module):
  def __init__(self, d_in, d_out, qkv = False):
    super().__init__()
    self.w_query = torch.nn.Linear(d_in, d_out, bias=qkv)
    self.w_key = torch.nn.Linear(d_in, d_out, bias=qkv)
    self.w_value = torch.nn.Linear(d_in, d_out, bias=qkv)

  def forward(self, x):

    query = self.w_query(inputs)
    key = self.w_key(inputs)
    value = self.w_value(inputs)

    attn_score = query @ key.T
    attn_weights = torch.softmax(attn_score / (key.shape[1] ** 0.5), dim=-1)
    context_vec = attn_weights @ value

    return context_vec

torch.manual_seed(123)

s1 = SelfAttentionv2(d_in, d_out)

s1(inputs)

tensor([[-0.5337, -0.1051],
        [-0.5323, -0.1080],
        [-0.5323, -0.1079],
        [-0.5297, -0.1076],
        [-0.5311, -0.1066],
        [-0.5299, -0.1081]], grad_fn=<MmBackward0>)

Compact casual attention

In [18]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape)

torch.Size([2, 6, 3])


In [19]:
class CausalAttention(nn.Module):

    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout) # New
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1)) # New

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2) # Changed transpose
        attn_scores.masked_fill_(  # New, _ ops are in-place
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)  # `:num_tokens` to account for cases where the number of tokens in the batch is smaller than the supported context_size
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights) # New

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)

context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)

context_vecs = ca(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


In [20]:
class Multiheadattention(nn.Module):
  def __init__(self, d_in, d_out, context_length, dropout,num_heads=2, qkv_bias=False):
    super().__init__()
    # The output dimension for each head should be d_out // num_heads
    # assuming d_out is the total desired output dimension of the multi-head attention
    if d_out % num_heads != 0:
        raise ValueError(
            f"d_out ({d_out}) must be divisible by num_heads ({num_heads})"
        )
    head_dim = d_out // num_heads

    self.heads = nn.ModuleList([
        CausalAttention(d_in, head_dim, context_length, dropout, qkv_bias)
        for _ in range(num_heads)
    ])

  def forward(self, x):
    # Concatenating outputs from each head will result in the total d_out dimension
    return torch.cat([h(x) for h in self.heads], dim=-1)

In [21]:
gpt_config = {
    "vocab_size": 50257,
    "context_length": 256,
    "n_embd": 768,
    "n_head": 12,
    "n_layer": 12,
    "dropout": 0.1,
    "bias": False
}


In [22]:
import torch
import torch.nn as nn


class DummyGPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["n_embd"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["n_embd"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        # Use a placeholder for TransformerBlock
        self.trf_blocks = nn.Sequential(
            *[DummyTransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        # Use a placeholder for LayerNorm
        self.final_norm = DummyLayerNorm(cfg["n_embd"])
        self.out_head = nn.Linear(
            cfg["n_embd"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits


class DummyTransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        # A simple placeholder

    def forward(self, x):
        # This block does nothing and just returns its input.
        return x


class DummyLayerNorm(nn.Module):
    def __init__(self, normalized_shape, eps=1e-5):
        super().__init__()
        # The parameters here are just to mimic the LayerNorm interface.

    def forward(self, x):
        # This layer does nothing and just returns its input.
        return x

In [23]:
import torch.nn as nn

class layernorm(nn.Module):
  def __init__(self, cfg, eps=1e-5):
    super().__init__()
    self.eps = eps
    self.weight = nn.Parameter(torch.ones(cfg['n_embd']))
    self.bias = nn.Parameter(torch.zeros(cfg['n_embd']))

  def forward(self, x):
    mean = x.mean(-1, keepdim=True)
    var = x.var(-1, keepdim=True, unbiased=False)
    norm = (x - mean) / torch.sqrt(var + self.eps)
    return norm * self.weight + self.bias


# gelu activation

class gelu(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self, x):
    return 0.5 * x * (1 + torch.tanh(
        torch.sqrt(torch.tensor(2 / torch.pi)) * (x + 0.044715 * torch.pow(x, 3))
    ))


# feedforward

class feedforward(nn.Module):
  def __init__(self, cfg):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Linear( cfg['n_embd'], 4 * cfg['n_embd']),
        gelu(),
        nn.Linear(4 * cfg['n_embd'], cfg['n_embd'])
    )

  def forward(self, x):
    return self.layers(x)

In [24]:
class transformerblock(nn.Module):
  def __init__(self, cfg):
    super().__init__()
    self.att = Multiheadattention(
        d_in=cfg['n_embd'],
        d_out=cfg['n_embd'],
        context_length=cfg['context_length'],
        dropout=cfg['dropout'],
        num_heads=cfg['n_head'],
        qkv_bias=cfg['bias']
    )
    self.ln1 = layernorm(cfg)
    self.ff = feedforward(cfg)
    self.ln2 = layernorm(cfg)
    self.dropout = nn.Dropout(cfg['dropout'])


  def forward(self, x):

    shortcut = x
    x = self.ln1(x)
    x = self.att(x)
    x = self.dropout(x)
    x = x + shortcut

    shortcut = x
    x = self.ln2(x) # Corrected: should be assignment to self.ln2
    x = self.ff(x)
    x = self.dropout(x)
    x = x + shortcut

    return x

In [25]:
torch.manual_seed(123)

x = torch.randn(2, 3, 768)
block = transformerblock(gpt_config)
output = block(x)

print(output.shape)

torch.Size([2, 3, 768])


In [26]:
import torch
import torch.nn as nn


class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["n_embd"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["n_embd"])
        self.drop_emb = nn.Dropout(cfg["dropout"])

        self.trf_blocks = nn.Sequential(
            *[transformerblock(cfg) for _ in range(cfg["n_layer"])])

        self.final_norm = layernorm(cfg)
        self.out_head = nn.Linear(
            cfg["n_embd"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

In [27]:
torch.manual_seed(123)

gpt = GPTModel(gpt_config)


x = torch.randint(0, gpt_config['vocab_size'], (2, gpt_config['context_length']))

output = gpt(x)

print(output.shape)

torch.Size([2, 256, 50257])


In [28]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

batch = []

text1 = "every effort moves you"
text2 = "every day holds a"

batch.append(tokenizer.encode(text1))
batch.append(tokenizer.encode(text2))

batch = torch.tensor(batch)

print(batch)

tensor([[16833,  3626,  6100,   345],
        [16833,  1110,  6622,   257]])


In [29]:
ouput1 = gpt(batch)

ouput1.shape

torch.Size([2, 4, 50257])

In [30]:
start = 'hello, i am'

encoded = tokenizer.encode(start)

encoded = torch.tensor(encoded).unsqueeze(0)

print(encoded.shape)

torch.Size([1, 4])


In [31]:
def generate(model, idx, max_new_tokens, context_length):
  for _ in range(max_new_tokens):
    idx_con = idx[: , -context_length:]

    with torch.no_grad():
      logits = model(idx_con)
    logits = logits[:, -1, :]
    probas = torch.softmax(logits, dim=-1)
    idx_next = torch.multinomial(probas, num_samples=1)
    idx = torch.cat((idx, idx_next), dim=1)
  return idx

In [32]:
out = generate(gpt, encoded, 5, gpt_config['context_length'])

decodeded = tokenizer.decode(out[0].tolist())

print(decodeded)


hello, i amivers qualifications Ch||||Republic


In [33]:
import tiktoken
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

#tokenizer and dataloder.


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True, num_workers=0):
    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)

    return dataloader


#masked mulithead atttention.

class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by n_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads  # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x)  # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.reshape(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)  # optional projection

        return context_vec


#####################################
# Chapter 4
#####################################
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift


class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) *
            (x + 0.044715 * torch.pow(x, 3))
        ))


class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)


class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"])
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        # Shortcut connection for attention block
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)   # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        # Shortcut connection for feed-forward block
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        return x

#gptmodel.

class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits


def generate_text_simple(model, idx, max_new_tokens, context_size):
    # idx is (B, T) array of indices in the current context
    for _ in range(max_new_tokens):

        # Crop current context if it exceeds the supported context size
        # E.g., if LLM supports only 5 tokens, and the context size is 10
        # then only the last 5 tokens are used as context
        idx_cond = idx[:, -context_size:]

        # Get the predictions
        with torch.no_grad():
            logits = model(idx_cond)

        # Focus only on the last time step
        # (batch, n_token, vocab_size) becomes (batch, vocab_size)
        logits = logits[:, -1, :]

        # Get the idx of the vocab entry with the highest logits value
        idx_next = torch.argmax(logits, dim=-1, keepdim=True)  # (batch, 1)

        # Append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch, n_tokens+1)

    return idx


if __name__ == "__main__":

    GPT_CONFIG_124M = {
        "vocab_size": 50257,     # Vocabulary size
        "context_length": 256,  # Context length
        "emb_dim": 768,          # Embedding dimension
        "n_heads": 12,           # Number of attention heads
        "n_layers": 12,          # Number of layers
        "drop_rate": 0.1,        # Dropout rate
        "qkv_bias": False        # Query-Key-Value bias
    }

    torch.manual_seed(123)
    model = GPTModel(GPT_CONFIG_124M)
    model.eval()  # disable dropout

    start_context = "Hello, I am"

    tokenizer = tiktoken.get_encoding("gpt2")
    encoded = tokenizer.encode(start_context)
    encoded_tensor = torch.tensor(encoded).unsqueeze(0)

    print(f"\n{50*'='}\n{22*' '}IN\n{50*'='}")
    print("\nInput text:", start_context)
    print("Encoded input text:", encoded)
    print("encoded_tensor.shape:", encoded_tensor.shape)

    out = generate_text_simple(
        model=model,
        idx=encoded_tensor,
        max_new_tokens=10,
        context_size=GPT_CONFIG_124M["context_length"]
    )
    decoded_text = tokenizer.decode(out.squeeze(0).tolist())

    print(f"\n\n{50*'='}\n{22*' '}OUT\n{50*'='}")
    print("\nOutput:", out)
    print("Output length:", len(out[0]))
    print("Output text:", decoded_text)


                      IN

Input text: Hello, I am
Encoded input text: [15496, 11, 314, 716]
encoded_tensor.shape: torch.Size([1, 4])


                      OUT

Output: tensor([[15496,    11,   314,   716, 13240, 11381,  4307,  7640, 16620, 34991,
          6842, 37891, 19970, 47477]])
Output length: 14
Output text: Hello, I am Laur inhab DistrinetalkQueue bear confidentlyggyenium


In [34]:
inputs = torch.tensor([[16833, 3626, 6100],
                       [40, 1107, 588]])

targets = torch.tensor([[3626, 6100, 345],
                        [1107, 588, 11311]])

In [35]:
with torch.no_grad():
  logits = model(inputs)

In [36]:
logits.shape

torch.Size([2, 3, 50257])

In [37]:
probas = torch.softmax(logits, dim=-1)
probas.shape

torch.Size([2, 3, 50257])

In [38]:
token_ids = torch.argmax(probas, dim=-1, keepdim=True)
token_ids

tensor([[[16657],
         [  339],
         [42826]],

        [[49906],
         [29669],
         [41751]]])

In [39]:
print(tokenizer.decode(token_ids[0].squeeze().tolist()))
print(tokenizer.decode(targets[0].tolist()))

 Armed heNetflix
 effort moves you


In [40]:
logits_flat = logits.flatten(0,1)
targets_flat = targets.flatten()

print(logits_flat.shape)
print(targets_flat.shape)


torch.Size([6, 50257])
torch.Size([6])


In [41]:
torch.nn.functional.cross_entropy(logits_flat, targets_flat)

tensor(10.7940)

In [44]:
import os
import urllib.request

url = "https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/01_main-chapter-code/the-verdict.txt"

with urllib.request.urlopen(url) as response:
  text_data = response.read().decode('utf-8')

In [45]:
text_data

'\n\n\n\n\n\n<!DOCTYPE html>\n<html\n  lang="en"\n  \n  data-color-mode="auto" data-light-theme="light" data-dark-theme="dark"\n  data-a11y-animated-images="system" data-a11y-link-underlines="true"\n  \n  >\n\n\n\n\n  <head>\n    <meta charset="utf-8">\n  <link rel="dns-prefetch" href="https://github.githubassets.com">\n  <link rel="dns-prefetch" href="https://avatars.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">\n  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">\n  <link rel="preconnect" href="https://github.githubassets.com" crossorigin>\n  <link rel="preconnect" href="https://avatars.githubusercontent.com">\n\n  \n\n\n  <link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/light-dac525bbd821.css" /><link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/light_high_contrast-56ccf4057897.css" /><link crossorig